In [1]:
import requests
from bs4 import BeautifulSoup
import time
from urllib import urlencode
import selenium.webdriver
import random
from pymongo import MongoClient
import json
import boto3
import pandas as pd
from parse_clean_store import (parse_route_page, parse_user)
from bs4 import BeautifulSoup
from matrix_recommender_prep import (clean_ratings, cold_start)
%load_ext autoreload
%autoreload 2

ImportError: No module named matrix_recommender_prep

In [2]:
client = MongoClient('mongodb://localhost:27017/')
db = client.route_html_collection
route_html_collection = db.route_html_collection
raw_data = route_html_collection.find()
df_route_html = pd.DataFrame(list(raw_data))
df_route_html.head()

,_id,html,route
0,58c87f8840b44102a84aedf9,"<!DOCTYPE html><html xmlns=""http://www.w3.org/...",30(1) Feet of Pleasure
1,58c87f9740b44102a84aedff,"<!DOCTYPE html><html xmlns=""http://www.w3.org/...",41 Feet of Pain
2,58c87fa640b44102a84aee05,"<!DOCTYPE html><html xmlns=""http://www.w3.org/...",5.5 Crack
3,58c87fb140b44102a84aee0b,"<!DOCTYPE html><html xmlns=""http://www.w3.org/...",5.6 crack
4,58c87fbc40b44102a84aee11,"<!DOCTYPE html><html xmlns=""http://www.w3.org/...",5.6 Dihedral


In [3]:
df_route_html.shape

(3626, 3)

In [4]:
client = MongoClient('mongodb://localhost:27017/')
db = client.user_html_collection
user_html_collection = db.user_html_collection
raw_data = user_html_collection.find()
df_user_html = pd.DataFrame(list(raw_data))
df_user_html.head()

,_id,html,username
0,58c87f9340b44102a84aedfd,"<!DOCTYPE html><html xmlns=""http://www.w3.org/...",adam winslow
1,58c87fa440b44102a84aee03,"<!DOCTYPE html><html xmlns=""http://www.w3.org/...",adam winslow
2,58c87fae40b44102a84aee09,"<!DOCTYPE html><html xmlns=""http://www.w3.org/...",Oscar Edge
3,58c87fb640b44102a84aee0f,"<!DOCTYPE html><html xmlns=""http://www.w3.org/...",Nick Drake
4,58c87fc340b44102a84aee15,"<!DOCTYPE html><html xmlns=""http://www.w3.org/...",Chris Culver


In [2]:
client = MongoClient('mongodb://localhost:27017/')
db = client.ratings_collection
ratings_collection = db.ratings_collection
raw_data = ratings_collection.find()
df_ratings = pd.DataFrame(list(raw_data))
df_ratings.head()

,_id,rating,route,username
0,58c87f8c40b44102a84aedfb,[1],30(1) Feet of Pleasure,[adam winslow ]
1,58c87f9d40b44102a84aee01,[1],41 Feet of Pain,[adam winslow ]
2,58c87fac40b44102a84aee07,"[1, 1, 1, 1]",5.5 Crack,"[CHopwood , Trevor. , Rickedge , Oscar Edge ]"
3,58c87fb440b44102a84aee0d,"[3, 3, 3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ...",5.6 crack,"[Bianca P , Sandeep Kumar , Mike Brady , Tom B..."
4,58c87fbf40b44102a84aee13,"[2, 2, 2, 2, 1, 1, 1, 1, 1]",5.6 Dihedral,"[Jason Killgore , sqwirll , Dave Leydet , fort..."


In [4]:
df_ratings.shape

(3726, 4)

In [13]:
ratings_df = clean_ratings(df_ratings)
ratings_df.head()

,adam winslow,CHopwood,Trevor.,Rickedge,Oscar Edge,Bianca P,Sandeep Kumar,Mike Brady,Tom Bath,Clay Studer,...,Andrew Perkins,Mike dandy Patz,Travis England,Neilsonb,Ben Broche,Dustin Stephens,Curt Shannon,Cedar Wright,Detchy,Colin Moorhead
30(1) Feet of Pleasure,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
41 Feet of Pain,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5.5 Crack,NaN,1.0,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5.6 crack,NaN,NaN,NaN,NaN,NaN,3.0,3.0,3.0,2.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5.6 Dihedral,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [19]:
ratings_df = ratings_df.T
ratings_df.head()

,30(1) Feet of Pleasure,41 Feet of Pain,5.5 Crack,5.6 crack,5.6 Dihedral,5.7,5.7 Face,A,A Crack,A MIGHTY WIND,...,TPMV (10% Meteorological Vinculation),Tradical,Tree Hugger,Trigger Finger,Trobability Pest,Twelve Gauge IQ,Under Arrest,Under the Influence,Unicorn Blues,Unknown
adam winslow,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
CHopwood,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Trevor.,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Rickedge,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Oscar Edge,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [61]:
cold_start(ratings_df).head()

,30(1) Feet of Pleasure,41 Feet of Pain,5.5 Crack,5.6 crack,5.6 Dihedral,5.7,5.7 Face,A,A Crack,A MIGHTY WIND,...,Town Crier,TPMV (10% Meteorological Vinculation),Tradical,Tree Hugger,Trigger Finger,Trobability Pest,Twelve Gauge IQ,Under Arrest,Under the Influence,Unicorn Blues
adam winslow,1.0,1.0,1.0,2.029412,1.444444,2.0,1.692308,NaN,1.0,3.0,...,3.545455,3.928571,4.0,3.0,4.0,3.0,4.0,2.615385,2.0,4.0
CHopwood,1.0,1.0,1.0,2.029412,1.444444,2.0,1.692308,NaN,1.0,3.0,...,3.545455,3.928571,4.0,3.0,4.0,3.0,4.0,2.615385,2.0,4.0
Trevor.,1.0,1.0,1.0,2.029412,1.444444,2.0,1.692308,NaN,1.0,3.0,...,3.545455,3.928571,4.0,3.0,4.0,3.0,4.0,2.615385,2.0,4.0
Rickedge,1.0,1.0,1.0,2.029412,1.444444,2.0,1.692308,NaN,1.0,3.0,...,3.545455,3.928571,4.0,3.0,4.0,3.0,4.0,2.615385,2.0,4.0
Oscar Edge,1.0,1.0,1.0,2.029412,1.444444,2.0,1.692308,NaN,1.0,3.0,...,3.545455,3.928571,4.0,3.0,4.0,3.0,4.0,2.615385,2.0,4.0


In [94]:
df = cold_start(df_ratings)
df.head()

,30(1) Feet of Pleasure,41 Feet of Pain,5.5 Crack,5.6 crack,5.6 Dihedral,5.7,5.7 Face,A,A Crack,A MIGHTY WIND,...,Town Crier,TPMV (10% Meteorological Vinculation),Tradical,Tree Hugger,Trigger Finger,Trobability Pest,Twelve Gauge IQ,Under Arrest,Under the Influence,Unicorn Blues
adam winslow,1.0,1.0,1.0,2.029412,1.444444,2.0,1.692308,NaN,1.0,3.0,...,3.545455,3.928571,4.0,3.0,4.0,3.0,4.0,2.615385,2.0,4.0
CHopwood,1.0,1.0,1.0,2.029412,1.444444,2.0,1.692308,NaN,1.0,3.0,...,3.545455,3.928571,4.0,3.0,4.0,3.0,4.0,2.615385,2.0,4.0
Trevor.,1.0,1.0,1.0,2.029412,1.444444,2.0,1.692308,NaN,1.0,3.0,...,3.545455,3.928571,4.0,3.0,4.0,3.0,4.0,2.615385,2.0,4.0
Rickedge,1.0,1.0,1.0,2.029412,1.444444,2.0,1.692308,NaN,1.0,3.0,...,3.545455,3.928571,4.0,3.0,4.0,3.0,4.0,2.615385,2.0,4.0
Oscar Edge,1.0,1.0,1.0,2.029412,1.444444,2.0,1.692308,NaN,1.0,3.0,...,3.545455,3.928571,4.0,3.0,4.0,3.0,4.0,2.615385,2.0,4.0


In [95]:
from sklearn.decomposition import NMF


In [96]:
df_user_html.shape

NameError: name 'df_user_html' is not defined

In [97]:
soup = BeautifulSoup(html, 'html.parser')
page_tag = soup.find('div', {'id':'rspCol800'})
# make route dict
route_dict = {}
route_dict['name'] = page_tag.find('span', {'itemprop':'itemreviewed'}).text.encode('utf-8')
route_dict['grade'] = page_tag.find('span', {'class':'rateYDS'}).text
route_stars_text = soup.find('span', {'id':'starSummaryText'}).text.split('Average: ')
route_dict['average_rating'] = route_stars_text[1][:3]

NameError: name 'html' is not defined

In [98]:
def parse_route_page(html):
    '''
    takes a soup to parse
    returns dict of route info
    '''
    soup = BeautifulSoup(html, 'html.parser')
    page_tag = soup.find('div', {'id':'rspCol800'})
    # make route dict
    route_dict = {}
    route_dict['name'] = page_tag.find('span', {'itemprop':'itemreviewed'}).text
    route_dict['grade'] = clean_grade(page_tag)
    route_dict['average_rating'] = clean_average_rating(soup)
    
    for i, td in enumerate(page_tag.find('table').findAll('td')): 
        if td.text.split(':')[0] == 'Type':
            route_type = clean_route_type(page_tag.find('table').findAll('td')[i+1])
            route_dict['route_type_list'] = route_type[0]
            route_dict['pitches'] = route_type[1]
            route_dict['height'] = route_type[2]
            route_dict['uiaa'] = route_type[3]
            route_dict['misc'] = route_type[4]
        elif td.text.split(':')[0] == 'Original':
            route_dict['original_grade'] = clean_original_grade(page_tag.find('table').findAll('td')[i+1])
        elif td.text.split(':')[0] == 'FA':
            fa = page_tag.find('table').findAll('td')[i+1].text
            route_dict['FA'] = fa
        elif td.text.split(':')[0] == 'Season':
            season = page_tag.find('table').findAll('td')[i+1].text
            route_dict['season'] = season.encode('utf-8')
        elif td.text.split(':')[0] == 'Page Views':
            page_view = page_tag.find('table').findAll('td')[i+1].text
            route_dict['page_views'] = int(page_view.encode('utf-8').replace(',',''))
        elif td.text.split(':')[0] == 'Submitted By':
            sb1, sb2 = clean_submitted_by(page_tag.find('table').findAll('td')[i+1])
            route_dict['submitted_by'] = sb1
            route_dict['submitted_on'] = sb2
            
    return route_dict


def clean_route_type(tag):
    route_type = tag.text
    route_type = route_type.encode('utf-8').split(', ')
    list_of_types = ['Trad', 'Sport', 'TR', 'Alpine', 
                    'Ice', 'Boulder', 'Aid', 'Mixed']
    # initialize variables
    route_type_list = []
    pitches = float('nan')
    height = float('nan')
    uiaa = float('nan')
    misc = float('nan')
    for item in route_type:
        if item in list_of_types:
            route_type_list.append(item)
        elif 'pitches' in item:
            pitches = item.replace(' pitches', '')
        elif 'pitch' in item:
            pitches = item.replace(' pitch','')
        elif "'" in item:
            height = float(item.replace("'",''))
        elif 'Grade' in item:
            uiaa = item
        else:
            misc = item

    return route_type_list, pitches, height, uiaa, misc


def clean_submitted_by(page_tag):
    sb = page_tag.text
    sb = sb.encode('utf-8').split(' on ')
    return sb[0], sb[1]

def clean_original_grade(page_tag):
    grade = page_tag.text
    grade = grade.encode('utf-8').split('\xa0')[1]
    grade = grade.split('French')[0]
    return grade

def clean_grade(page_tag):
    grade = page_tag.find('span', {'class':'rateYDS'}).text
    grade = grade.encode('utf-8').split('\xa0')[1]
    return grade

def clean_average_rating(soup):
    route_stars_text = soup.find('span', {'id':'starSummaryText'}).text.split('Average: ')
    average_rating = route_stars_text[1][:3]
    if 'OK' in average_rating:
        average_rating = 1.0
    elif 's' in average_rating:
        average_rating = float(average_rating[0])
    else:
        average_rating = float(average_rating)
    return average_rating

In [21]:
df_route_html['_id'].shape == df_route_html['_id'].unique().shape

True

In [3]:
dict_list = []
for _id, html in enumerate(df_route_html['html']):
    dict_list.append(parse_route_page(_id, html))

In [23]:
df_routes = pd.DataFrame(dict_list)
df_routes.head(50)

,Aid,Alpine,Boulder,FA,Ice,Mixed,Sport,TR,Trad,average_rating,...,height,id,name,original_grade,page_views,pitches,season,submitted_by,submitted_on,uiaa
0,0,0,0,"Adam Winslow, Bill Coe, Kyle Silverman 3/20/09",0,0,0,0,1,1.0,...,41.0,0,30(1) Feet of Pleasure,5.6,588,1.0,NaN,adam winslow,"Aug 18, 2010",NaN
1,0,0,0,"Adam Winslow, Ujahn Davisson, Bill Coe, Jim Op...",0,0,0,0,1,1.0,...,41.0,1,41 Feet of Pain,5.5,642,1.0,NaN,adam winslow,"Aug 18, 2010",NaN
2,0,0,0,?,0,0,0,1,1,1.0,...,35.0,2,5.5 Crack,5.5,176,NaN,Afternoon shade,CHopwood,"Sep 2, 2013",NaN
3,0,0,0,,0,0,0,1,1,2.0,...,85.0,3,5.6 crack,5.6,1263,1.0,NaN,Tom Bath,"Dec 16, 2009",NaN
4,0,0,0,Unknown,0,0,0,1,1,1.4,...,30.0,4,5.6 Dihedral,5.6,678,1.0,NaN,chrishar Hargarten,"May 19, 2008",NaN
5,0,0,0,,0,0,1,0,0,2.0,...,40.0,5,5.7,5.7,326,NaN,NaN,Keenan Waeschle,"Mar 31, 2010",NaN
6,0,0,0,,0,0,0,1,1,1.7,...,95.0,6,5.7 Face,5.7,454,1.0,NaN,Mark Goodro,"Jun 18, 2013",NaN
7,0,0,0,,0,0,1,0,0,2.0,...,95.0,7,A,5.5,18,NaN,NaN,applewood,"May 28, 2013",NaN
8,0,0,0,,0,0,0,0,1,1.0,...,NaN,8,A Crack,5.7,51,NaN,NaN,Blitzo,"Oct 18, 2006",NaN
9,0,0,0,10/13,0,0,1,0,0,3.0,...,75.0,9,A MIGHTY WIND,5.3,39,NaN,NaN,applewood,"Oct 27, 2013",NaN


In [4]:
client = MongoClient('mongodb://localhost:27017/')
db = client.routes
routes = db.routes
routes.insert_many(dict_list)

In [12]:
raw = routes.find()
pd.DataFrame(list(raw)).head()

,Aid,Alpine,Boulder,FA,Ice,Mixed,Sport,TR,Trad,_id,...,grade,height,name,original_grade,page_views,pitches,season,submitted_by,submitted_on,uiaa
0,0,0,0,"Adam Winslow, Bill Coe, Kyle Silverman 3/20/09",0,0,0,0,1,58cc183e40b441224553a5f2,...,5.6,41.0,30(1) Feet of Pleasure,5.6,588,1.0,NaN,adam winslow,"Aug 18, 2010",NaN
1,0,0,0,"Adam Winslow, Ujahn Davisson, Bill Coe, Jim Op...",0,0,0,0,1,58cc183e40b441224553a5f3,...,5.5,41.0,41 Feet of Pain,5.5,642,1.0,NaN,adam winslow,"Aug 18, 2010",NaN
2,0,0,0,?,0,0,0,1,1,58cc183e40b441224553a5f4,...,5.5,35.0,5.5 Crack,5.5,176,NaN,Afternoon shade,CHopwood,"Sep 2, 2013",NaN
3,0,0,0,,0,0,0,1,1,58cc183e40b441224553a5f5,...,5.6,85.0,5.6 crack,5.6,1263,1.0,NaN,Tom Bath,"Dec 16, 2009",NaN
4,0,0,0,Unknown,0,0,0,1,1,58cc183e40b441224553a5f6,...,5.6,30.0,5.6 Dihedral,5.6,678,1.0,NaN,chrishar Hargarten,"May 19, 2008",NaN


In [56]:
df_routes.shape[0]

3626

In [57]:
df_routes['name'].unique().shape[0]

3463

In [85]:
unique_names = df_user_html['username'].unique().tolist()

In [86]:
type(unique_names)

list

In [88]:
dict_list = []
for html, name in zip(df_user_html['html'], df_user_html['username']):
    if name in unique_names:
        dict_list.append(parse_user(html))
        unique_names.remove(name)
df_users = pd.DataFrame(dict_list)
df_users.head(50)

,Aid:,Favorite Climbs:,Ice:,Likes to climb:,Other Interests:,Personal:,Sport:,Trad:,compliments,last_vist,member_since,name,point_rank,total_points
0,NaN,none specified,NaN,"Trad, Sport, TR climbs",none specified,"Lives in Pullman WA, 14 years old, Male",Leads 5.11a 6c 22 VII+ 22 E3 5c Follows 5...,Leads 5.8 5b 16 VI- 15 HVS 4c Follows 5.1...,,"Jan 30, 2017","Jun 21, 2016",Oscar Edge,"7,891",31
1,NaN,"NR Stuart, Rattletail, Outer Space, anything ...",Leads WI2 Follows WI5,"Trad, Sport, Gym climbs",Splitboarding,"Lives in Newcastle, WA, 36 years old, Male",Leads 5.11a 6c 22 VII+ 22 E3 5c Follows 5...,Leads 5.10a 6a 18 VI+ 18 E1 5a Follows 5....,,3 hours ago,"Jan 20, 2015",Nick Drake,"1,812",353
2,NaN,none specified,NaN,none specified,none specified,NaN,NaN,NaN,0,1 day ago,"Jul 7, 2014",Chris Culver,-none-,0
3,NaN,none specified,NaN,"Trad, Sport, TR, Gym climbs",none specified,"Lives in Silverdale, WA, Male",NaN,Leads 5.11b 6c 23 VIII- 23 E3 5c Follows ...,2,"Jun 4, 2015","Oct 11, 2010",Ethan Henderson,"1,084",630
4,NaN,none specified,Leads WI4 Follows WI5,"Trad, Sport climbs",none specified,"Lives in Soldotna, Alaska, 44 years old, Male",Leads 5.10a 6a 18 VI+ 18 E1 5a Follows 5....,Leads 5.10a 6a 18 VI+ 18 E1 5a Follows 5....,,3 days ago,"Aug 21, 2014",Chris Walden,"1,764",367
5,NaN,none specified,NaN,none specified,none specified,"Lives in Tonasket, WA",NaN,NaN,,6 hours ago,"Aug 30, 2012",applewood,20,"11,385"
6,NaN,none specified,NaN,"Trad, Sport climbs","Martial arts, Herpetology, cooking","Lives in Everett, WA, 31 years old, Male",Leads 5.12a 7a+ 25 VIII+ 25 E5 6a Follows ...,Leads 5.10a 6a 18 VI+ 18 E1 5a Follows 5....,,9 hours ago,"Jul 14, 2015",Kyle Elliott,"2,066",295
7,NaN,none specified,NaN,"Trad, Sport, TR, Gym climbs",none specified,"Lives in North Bend, WA, 29 years old, Male",Leads 3rd 1- 1 I 1 M 1a Follows 4th 1 2 I...,Leads 3rd 1- 1 I 1 M 1a Follows 3rd 1- 1 ...,,7 hours ago,"Aug 4, 2013",Ryan Canfield,"2,535",225
8,NaN,Any alpine rock,NaN,"Trad, Sport, TR, Gym climbs","Mountaineering, skiing (alpine, tele, BC, XC,...",NaN,Leads 5.10c 6b 20 VII 20 E2 5b Follows 5....,Leads 5.8 5b 16 VI- 15 HVS 4c Follows 5.1...,,"Nov 12, 2016","May 12, 2016",Diana Y,"5,151",75
9,NaN,none specified,NaN,"Trad, Sport climbs",none specified,"Lives in Orting, WA, 26 years old, Male",Leads 5.1 2 6 II 7 MD 2a Follows 3rd 1- 1...,Leads 5.11a 6c 22 VII+ 22 E3 5c Follows 5...,3,2 days ago,"Feb 28, 2014",George Zack,"2,288",260


In [89]:
df_users['name'].shape[0]

680